In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression                                       #linear regression
from sklearn.preprocessing import PolynomialFeatures                                    #polynomial regression
from sklearn.svm import SVR                                                             #simple vector regression
from sklearn.tree import DecisionTreeRegressor                                          #decision tree regressor
from sklearn.ensemble import RandomForestRegressor                                      #random forest regressor
import gurobipy as gp
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [60]:
dataset = pd.read_csv('input/adatb_setcover_output.csv')

In [61]:
dataset.head()

,policy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime,objval
0,internal:relpscost,0,small,/data/IP-DL/instances/becsogergely/adatb/setco...,1,2,0.03,0.0,optimal,0,0,0.041468,0.041470,28.0
1,internal:relpscost,0,small,/data/IP-DL/instances/becsogergely/adatb/setco...,1,8,0.12,0.0,optimal,0,0,0.117496,0.117499,25.0
2,internal:relpscost,0,small,/data/IP-DL/instances/becsogergely/adatb/setco...,1,3,0.03,0.0,optimal,0,0,0.032750,0.032753,10.0
3,internal:relpscost,0,small,/data/IP-DL/instances/becsogergely/adatb/setco...,1,2,0.05,0.0,optimal,0,0,0.046181,0.046185,23.0
4,internal:relpscost,0,small,/data/IP-DL/instances/becsogergely/adatb/setco...,1,1,0.01,0.0,optimal,0,0,0.014738,0.014741,11.0


In [62]:
%%capture
for i in range(5000):
    m = gp.read("./train/instance_"+str(i+1)+".lp")                          # reading the LP files
    A = m.getA()                                                             # creating sparse matrix from LP files
    B=A.toarray()                                                            # converting the sparse matrix to ndarray
    dataset.at[i,"instance"]=B                                               # writing this matrix to the dataframe
    

In [63]:
#dropping columns that seem irrelevant

dataset = dataset.drop('policy', axis=1)
dataset = dataset.drop('seed', axis=1)
dataset = dataset.drop('type', axis=1)
dataset = dataset.drop('status', axis=1)
dataset = dataset.drop('nnodes', axis=1)
#dataset = dataset.drop('nlps', axis=1)
dataset = dataset.drop('gap', axis=1)
dataset = dataset.drop('ndomchgs', axis=1)
dataset = dataset.drop('ncutoffs', axis=1)

In [64]:
%%capture
m = gp.read("./train/instance_2.lp")                          # reading the LP files
l=[]
for i in range(5000):
    m = gp.read("./train/instance_"+str(i+1)+".lp")
    c = m.getAttr("Obj",m.getVars())
    l=l+[c]


In [65]:
dataset["objective"]=l

In [66]:
dataset.head(10)

,instance,nlps,stime,walltime,proctime,objval,objective
0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,0.03,0.041468,0.041470,28.0,"[19.0, 91.0, 40.0, 51.0, 78.0, 3.0, 84.0, 78.0..."
1,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...",8,0.12,0.117496,0.117499,25.0,"[67.0, 4.0, 48.0, 73.0, 9.0, 99.0, 27.0, 61.0,..."
2,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3,0.03,0.032750,0.032753,10.0,"[22.0, 48.0, 71.0, 56.0, 16.0, 24.0, 17.0, 44...."
3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...",2,0.05,0.046181,0.046185,23.0,"[31.0, 88.0, 70.0, 27.0, 76.0, 45.0, 29.0, 37...."
4,"[[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0,...",1,0.01,0.014738,0.014741,11.0,"[62.0, 4.0, 53.0, 1.0, 46.0, 24.0, 50.0, 72.0,..."
5,"[[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0,...",26,0.24,0.250231,0.250223,17.0,"[8.0, 74.0, 68.0, 100.0, 45.0, 100.0, 11.0, 39..."
6,"[[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0,...",30,0.33,0.331593,0.331595,19.0,"[80.0, 14.0, 65.0, 47.0, 99.0, 3.0, 85.0, 74.0..."
7,"[[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0,...",37,0.35,0.367199,0.367201,30.0,"[90.0, 42.0, 45.0, 90.0, 9.0, 70.0, 41.0, 21.0..."
8,"[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,...",2,0.02,0.017138,0.017119,10.0,"[28.0, 15.0, 40.0, 19.0, 80.0, 11.0, 26.0, 36...."
9,"[[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0,...",1,0.02,0.015780,0.015783,32.0,"[76.0, 2.0, 18.0, 55.0, 32.0, 53.0, 16.0, 83.0..."


In [67]:
dataset_flat=dataset
l=['a0']
for i in range(19999):
    l = l + ['a'+str(i+1)]
dataset_flat = pd.concat(
    [
        dataset_flat,
        pd.DataFrame(
            np.empty([5000, 20000]), 
            index=dataset_flat.index, 
            columns=l
        )
    ], axis=1
)


for i in dataset_flat.index:
    m=dataset_flat.at[i, "instance"]
    mflat=m.flatten()
    for j in range(20000):
        dataset_flat.at[i,"a"+str(j)]=mflat[j]

In [68]:
dataset_flat.head()

,instance,nlps,stime,walltime,proctime,objval,objective,a0,a1,a2,...,a19990,a19991,a19992,a19993,a19994,a19995,a19996,a19997,a19998,a19999
0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,0.03,0.041468,0.041470,28.0,"[19.0, 91.0, 40.0, 51.0, 78.0, 3.0, 84.0, 78.0...",1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...",8,0.12,0.117496,0.117499,25.0,"[67.0, 4.0, 48.0, 73.0, 9.0, 99.0, 27.0, 61.0,...",1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3,0.03,0.032750,0.032753,10.0,"[22.0, 48.0, 71.0, 56.0, 16.0, 24.0, 17.0, 44....",0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...",2,0.05,0.046181,0.046185,23.0,"[31.0, 88.0, 70.0, 27.0, 76.0, 45.0, 29.0, 37....",0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
4,"[[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0,...",1,0.01,0.014738,0.014741,11.0,"[62.0, 4.0, 53.0, 1.0, 46.0, 24.0, 50.0, 72.0,...",0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
l2=['c0']
for i in range(199):
    l2=l2+["c"+str(i)]   
dataset_flat = pd.concat(
    [
        dataset_flat,
        pd.DataFrame(
            np.empty([5000, 200]), 
            index=dataset_flat.index, 
            columns=l2
        )
    ], axis=1
)

for i in dataset_flat.index:
    for j in range(200):
        dataset_flat.at[i,"c"+str(j)]=dataset_flat.at[i,"objective"][j]

In [ ]:
dataset_flat.head()

In [25]:
dataset_flat=dataset_flat.drop('objective', axis=1)
dataset_flat=dataset_flat.drop('instance', axis=1)

In [26]:
dataset_flat.head()

,stime,walltime,proctime,objval,a0,a1,a2,a3,a4,a5,...,c190,c191,c192,c193,c194,c195,c196,c197,c198,c199
0,0.03,0.041468,0.041470,28.0,1.0,0.0,0.0,0.0,0.0,0.0,...,55.0,80.0,28.0,54.0,30.0,31.0,16.0,88.0,47.0,32.0
1,0.12,0.117496,0.117499,25.0,1.0,0.0,0.0,0.0,0.0,0.0,...,97.0,28.0,17.0,98.0,18.0,79.0,49.0,2.0,50.0,1.0
2,0.03,0.032750,0.032753,10.0,0.0,0.0,1.0,0.0,0.0,0.0,...,45.0,38.0,30.0,59.0,96.0,33.0,29.0,81.0,82.0,85.0
3,0.05,0.046181,0.046185,23.0,0.0,1.0,0.0,0.0,0.0,0.0,...,19.0,75.0,30.0,92.0,17.0,20.0,10.0,19.0,95.0,28.0
4,0.01,0.014738,0.014741,11.0,0.0,0.0,1.0,1.0,0.0,1.0,...,57.0,78.0,34.0,53.0,44.0,67.0,86.0,1.0,16.0,60.0


In [31]:
dataset_flat.to_csv("flattened_dataset.csv")

In [32]:
train_ds=dataset[:4500]
test_ds=dataset[4500:]

In [43]:
#dataset_flat=pd.read_csv("flattened_dataset.csv")

In [67]:
X_train = dataset_flat
X_train = X_train.drop('stime', axis=1)
X_train = X_train.drop('walltime', axis=1)
X_train = X_train.drop('proctime', axis=1)
X_train = X_train.drop('objval', axis=1)
X_train = X_train.drop(X_train.columns[0], axis=1)
X_test = X_train[4500:]
X_train = X_train[:4500]
Y_train = dataset_flat['proctime']
Y_test = Y_train[4500:]
Y_train = Y_train[:4500]
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(4500, 20200) (4500,) (500, 20200) (500,)


In [58]:
X_train.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,c190,c191,c192,c193,c194,c195,c196,c197,c198,c199
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,55.0,80.0,28.0,54.0,30.0,31.0,16.0,88.0,47.0,32.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,97.0,28.0,17.0,98.0,18.0,79.0,49.0,2.0,50.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,45.0,38.0,30.0,59.0,96.0,33.0,29.0,81.0,82.0,85.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,19.0,75.0,30.0,92.0,17.0,20.0,10.0,19.0,95.0,28.0
4,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,57.0,78.0,34.0,53.0,44.0,67.0,86.0,1.0,16.0,60.0


In [61]:
Y_train.head()

0    0.041470
1    0.117499
2    0.032753
3    0.046185
4    0.014741
Name: proctime, dtype: float64